# Assignment - Basic Pandas
<sup>Created by Natawut Nupairoj, Department of Computer Engineering, Chulalongkorn University</sup>

Using pandas to explore youtube trending data from GB (GBvideos.csv and GB_category_id.json) and answer the questions.

In [2]:
import pandas as pd
import numpy as np
!pip install pandasql
import pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=494d060310c5a4a6581b677b84389a55327d1511f7d33c42d81c74aefe61c352
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


To simplify data retrieval process on Colab, we heck if we are in the Colab environment and download data files from a shared drive and save them in folder "data".

For those using jupyter notebook on the local computer, you can read data directly assuming you save data in the folder "data".

In [3]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !wget https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/data.tgz -O data.tgz
    !tar -xzvf data.tgz

--2022-01-17 09:09:32--  https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/data.tgz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kaopanboonyuen/2110446_DataScience_2021s2/main/datasets/data.tgz [following]
--2022-01-17 09:09:32--  https://raw.githubusercontent.com/kaopanboonyuen/2110446_DataScience_2021s2/main/datasets/data.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45477462 (43M) [application/octet-stream]
Saving to: ‘data.tgz’

data.tgz            100%[===================>]  43.37M   210MB/s    in 0.2s    

2022-01-17 09:09:33 (210 MB/s) - ‘data.tgz’ sa

## How many rows are there in the GBvideos.csv after removing duplications?

In [4]:
# read .csv
df = pd.read_csv('data/GBvideos.csv')
df.shape

(38916, 16)

In [68]:
# remove duplications
df.drop_duplicates(inplace=True)
df.shape
print(f'rows after removing duplications = {df.shape[0]}')

rows after removing duplications = 38745


## How many VDO that have "dislikes" more than "likes"?  Make sure that you count only unique title!

In [ ]:
df.info()

In [ ]:
df.head()

In [8]:
#df[df.dislikes > df.likes][['title']]

In [9]:
len(df[df.dislikes > df.likes]['title'].unique())

56

In [10]:
sql_df = pandasql.sqldf("SELECT DISTINCT title FROM df WHERE dislikes > likes;")
len(sql_df)

56

## How many VDO that are trending on 22 Jan 2018 with comments more than 10,000 comments?

In [11]:
len(df[(df.trending_date.str.contains("18.22.01")) & (df.comment_count > 10000)][['trending_date','comment_count','title']])

28

In [12]:
len(pandasql.sqldf("SELECT trending_date, comment_count, title FROM df WHERE trending_date == '18.22.01' AND comment_count > 10000;"))

28

## Which date that has the minimum average number of comments per VDO?

In [13]:
#df[df.trending_date == df.groupby('trending_date')['comment_count'].mean().min()]

In [14]:
pandasql.sqldf("SELECT trending_date, MIN(avr)                                   \
                FROM (                                                           \
                  SELECT trending_date, AVG(CAST(comment_count AS FLOAT)) AS avr\
                  FROM df                                                       \
                  GROUP BY trending_date) AS t")

,trending_date,MIN(avr)
0,17.15.11,4738.673367


## Compare "Sports" and "Comedy", how many days that there are more total daily views of VDO in "Sports" category than in "Comedy" category?

In [23]:
import json

with open('data/GB_category_id.json') as fd:
    cat = json.load(fd)

cat_list = []
for d in cat['items']:
    if (d['snippet']['title'] == 'Comedy' or d['snippet']['title'] == 'Sports' ):
      cat_list.append((int(d['id']), d['snippet']['title']))
cat_list

# comedy = 23, 34  
# sport = 17

[(17, 'Sports'), (23, 'Comedy'), (34, 'Comedy')]

In [62]:
pandasql.sqldf("\
SELECT sports.date, sports.sum AS 'total daily views (sports)', comedy.sum AS 'total daily views (comedy)' \
FROM ( \
  SELECT trending_date AS date, SUM(views) AS sum\
  FROM df \
  WHERE category_id == 17 \
  GROUP BY trending_date) AS sports, \
  (SELECT trending_date AS date, SUM(views) AS sum \
  FROM df \
  WHERE category_id == 23 OR category_id == 34  \
  GROUP BY trending_date) AS comedy \
WHERE sports.date == comedy.date AND sports.sum > comedy.sum;")

,date,total daily views (sports),total daily views (comedy)
0,18.01.03,55631958,17726084
1,18.01.04,22974582,20504068
2,18.02.03,55755300,18412641
3,18.02.04,23413439,21044420
4,18.03.03,56546489,19077034
5,18.03.04,23684218,22202105
6,18.04.02,8911374,6221294
7,18.04.03,57311658,8484440
8,18.05.02,10213544,6542434
9,18.05.03,57939026,9090302
